# Filtragem

In [ ]:
import numpy as np
from numpy.fft import fft, ifft, fftfreq
import matplotlib.pyplot as plt

Vamos definir um sinal simples, formado pela soma entre dois sinais senos com frequências distintas. Para ilustrar o nosso estudo de filtros, primeiramente vamos tentar isolar a componente de baixa de frequência do sinal, usando um filtro **passa baixa**.

In [ ]:
N = 201 # número de amostras
t = np.linspace(0, 1, N) # sinal definido entre 0 e 1 segundos
dt = t[1] - t[0] # intervalo de amostragem

In [ ]:
# frequência de Nyquist
f_ny = 1/(2*dt)
f_ny

In [ ]:
# definindo frequências de interesse
f1 = 5
f2 = 40

In [ ]:
# definindo séries temporais
x1 = 2*np.sin(2*np.pi*f1*t)
x2 = np.sin(2*np.pi*f2*t)

plt.plot(t, x1, label="x1")
plt.plot(t, x2, label="x2")
plt.xlabel("Tempo (s)")
plt.ylabel("Amplitude")
plt.legend();

In [ ]:
# vamos obter um sinal formado pela soma entre x1 e x2
x = x1+x2

plt.plot(t, x, 'k')
plt.xlabel("Tempo (s)")
plt.ylabel("Amplitude");

In [ ]:
# Para aplicar um filtro passa baixa, vamos levar o sinal
# para o domínio da frequência

x_FFT = fft(x)
freqs = fftfreq(N, dt)

plt.plot(freqs[:N//2], np.abs(x_FFT[:N//2]))
plt.xlabel("$f_n (s^{-1})$")
plt.ylabel("$|X_k|$")
plt.title("FFT")
plt.axvline(f_ny, ls="--", color="k");

O gráfico da Transformada de Fourier nos mostra dois picos centrados nas frequências de 5 e 40 Hz, ou seja, as frequências dos sinais x1 e x2 que definimos anteriormente. Perceba que ambas frequências estão abaixo do limite estabelecido pela frequência de Nyquist. Para aplicar o filtro passa baixa, vamos cortar as frequências maiores do que, por exemplo, 10 Hz.

In [ ]:
# indices com as frequências menores do que 10 Hz
indices = np.abs(freqs) < 10.0
# para filtrar, multiplicamos o sinal no dom. da frequência
# pelo vetor de índices
x_FFT_filt = x_FFT * indices
# por fim, voltamos o sinal para o domínio do tempo
x_filt = ifft(x_FFT_filt)

In [ ]:
print("Média (imag): %s"%np.mean(np.imag(x_filt)))
print("Mínimo (imag): %s"%np.min(np.imag(x_filt)))
print("Máximo (imag): %s"%np.max(np.imag(x_filt)))

Perceba que sobrou um resíduo complexo no sinal filtrado, mas são números muito pequenos, possivelmente um resíduo computacional. Vamos comparar o final filtrado recuperado com a sequência x1 definida no começo do notebook. Usaremos a parte real do sinal filtrado.

In [ ]:
plt.plot(t, x1, label="x1")
plt.plot(t, np.real(x_filt), label="x_filt")
plt.xlabel("Tempo (s)")
plt.ylabel("Amplitude")
plt.legend();

Utilizando o filtro passa baixa, obtivemos um sinal idêntico ao sinal x1 original.

**Exercício.** Considerando o sinal x definido anteriormente, aplique um filtro **passa alta** de forma a remover a componente de mais baixa frequência, ou seja, remova o sinal x1 de x. Plote o resultado.

Conseguimos separar os sinais simples muito bem, mas o que aconteceria no caso de os sinais estarem contaminados com ruído, que é o caso de dados reais? Vamos investigar a seguir, novamente tentando separar a componente de baixa frequência.

In [ ]:
N = 201 # número de amostras
t = np.linspace(0, 1, N) # sinal definido entre 0 e 1 segundos
dt = t[1] - t[0] # intervalo de amostragem

# frequência de Nyquist
f_ny = 1/(2*dt)
f_ny

In [ ]:
# definindo frequências de interesse
f1 = 5
f2 = 40

In [ ]:
# definindo séries temporais
x1_ruido = 2*np.sin(2*np.pi*f1*t) + 0.3*np.random.randn(len(t))
x2_ruido = np.sin(2*np.pi*f2*t) + 0.15*np.random.randn(len(t))

plt.plot(t, x1_ruido, label="x1_ruido")
plt.plot(t, x2_ruido, label="x2_ruido")
plt.xlabel("Tempo (s)")
plt.ylabel("Amplitude")
plt.legend();

In [ ]:
# vamos obter um sinal formado pela soma de x1 e x2
# e adicionar mais uma camada de ruído
x_ruido = x1_ruido + x2_ruido + 0.1*np.random.randn(len(t))

plt.plot(t, x_ruido, 'k')
plt.xlabel("Tempo (s)")
plt.ylabel("Amplitude");

In [ ]:
# aplicando a FFT
x_FFT_r = fft(x_ruido)
freqs = fftfreq(N, dt)

plt.plot(freqs[:N//2], np.abs(x_FFT_r[:N//2]))
plt.xlabel("$f_n (s^{-1})$")
plt.ylabel("$|X_k|$")
plt.title("FFT")
plt.axvline(f_ny, ls="--", color="k");

In [ ]:
# aplicando o filtro passa baixa

indices = np.abs(freqs) < 10.0
x_FFT_filt_r = x_FFT_r * indices

x_filt_r = ifft(x_FFT_filt_r)

In [ ]:
# plotando e comparando com x1
plt.plot(t, x1, 'r', label="x1")
plt.plot(t, x1_ruido, label="x1_ruido")
plt.plot(t, np.real(x_filt_r), label="x_filt_r")
plt.xlabel("Tempo (s)")
plt.ylabel("Amplitude")
plt.legend();

Dessa vez, o sinal filtrado é uma versão suavizada do sinal original, ou seja, muito mais parecido com o sinal x1 original do que com o x1 contaminado com ruído. Isso sugere que a FFT pode ser usada não somente para separar conteúdos de frequência de interesse dentro do sinal, mas também para amenizar o ruído. Vamos analisar um segundo exemplo nesse sentido, mas agora usando **espectro de potência**.

# Filtrando ruído

In [ ]:
# definindo sinais simples com duas frequências
dt = 0.001
t = np.arange(0, 1, dt)
x = np.sin(2*np.pi*50*t) + np.sin(2*np.pi*120*t)
x_r = x + 2.5*np.random.randn(len(t))

Perceba que criamos um sinal com bastante ruído, uma vez que o sinal original varia entre -2 e 2. Vamos plotar os dois.

In [ ]:
plt.plot(t, x_r, 'k', label="Ruidoso")
plt.plot(t, x, 'r', label="Limpo")
plt.xlabel("Tempo (s)")
plt.ylabel("Amplitude")
plt.legend();

Vamos supor que recebemos o sinal ruidoso e não sabemos que ele foi gerado a partir de dois sinais senos com frequências de 50 e 120 Hz.

In [ ]:
N = len(t)
x_fft = fft(x_r)
freqs = fftfreq(N, dt)
psd = x_fft * np.conj(x_fft) / N # espectro de potência
psd = np.real(psd) # ignorando uma componente 0j imaginária

plt.plot(freqs[:N//2], psd[:N//2], color="r", label="Ruidoso")
plt.xlabel("Frequência (Hz)")
plt.ylabel("Amplitude")
plt.title("Densidade espectral")
plt.legend();

Dito de forma simples, o eixo y do gráfico nos diz o quanto de "potência" há em cada frequência do sinal. Apesar de o sinal ser ruidoso, o espectro de potência tem dois picos muito claros, um em 50 Hz e outro em 120 Hz. Para filtrar, vamos cortar todos os coeficientes de Fourier menores do que, por exemplo, 100, e manter todos os maiores do que 100.

In [ ]:
indices = psd > 100 # indices onde psd > 100
psd_filt = psd * indices # filtrando o psd
x_fft_filt = x_fft * indices # filtrando os coef. de Fourier

# por fim, vamos obter a série temporal por meio da FFT inversa
x_filt = ifft(x_fft_filt)
x_filt = np.real(x_filt) # descartando resíduos complexos

In [ ]:
# vamos comparar os resultados

fig = plt.figure(figsize=(9, 6))
ax1 = fig.add_subplot(211)
ax1.plot(t, x_r, label="Ruidoso")
ax1.plot(t, x, label="Limpo")
ax1.plot(t, x_filt, label="Filtrado")
ax1.set_xlabel("Tempo (s)")
ax1.set_ylabel("Amplitude")
ax1.legend()

ax2 = fig.add_subplot(212)
ax2.plot(t, x, label="Limpo")
ax2.plot(t, x_filt, label="Filtrado")
ax2.set_xlabel("Tempo (s)")
ax2.set_ylabel("Amplitude")
ax2.legend()
plt.tight_layout();

Apesar de o ruído branco inserido possuir uma grande amplitude, conseguimos filtrá-lo com sucesso. Vamos plotar também os espectros de potência.

In [ ]:
fig = plt.figure(figsize=(9, 6))
ax1 = fig.add_subplot(211)
ax1.plot(freqs[:N//2], psd[:N//2], color="r", label="Ruidoso")
ax1.set_xlabel("Frequência (Hz)")
ax1.set_ylabel("Amplitude")
ax1.set_title("Densidade espectral")
ax1.legend()

ax2 = fig.add_subplot(212)
ax2.plot(freqs[:N//2], psd_filt[:N//2], color="r", label="Filtrado")
ax2.set_xlabel("Frequência (Hz)")
ax2.set_ylabel("Amplitude")
ax2.set_title("Densidade espectral")
ax2.legend()
plt.tight_layout();

# Trabalhando com dado real

Vamos analisar o registro de um terremoto que aconteceu em 06/09/2020 às 06:51:18, detectado pela estação AQDB. A biblioteca `Obspy` permite ler e realizar diversos processamentos com dados sismológicos, mas vamos utilizá-la apenas para ler os dados. 

In [ ]:
# pip install obspy
from obspy import read

In [ ]:
st = read("./Dados/sismograma.sac") # insira o caminho correto para o arquivo

In [ ]:
tr = st[0]

In [ ]:
# extraindo informações e criando vetores
x = tr.data
dt = tr.stats.delta
sr = tr.stats.sampling_rate
N = tr.stats.npts
t = np.arange(0, N/sr, dt)

In [ ]:
fig = plt.figure(figsize=(10, 6))
plt.plot(t, x, 'k')
plt.ylabel("Contagem")
plt.xlabel("Tempo desde a origem (s)");

**Exercício.** Plote a Transformada de Fourier do sinal e a seguir faça testes com filtros passa baixa, passa alta e passa banda.